In [86]:
import csv
import pandas as pd
import numpy as np
import os  
import requests    
from bs4 import BeautifulSoup 
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import snowball
import re
import string

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\himaj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\himaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
def webcrawler(initial_url, max_loop):
   
    write_to_file=[]  
    count= 0
    
    first_url=initial_url + '/publications/'

    Queue = [first_url]
    
    baseurl = "https://pureportal.coventry.ac.uk"
 
    while(Queue!=[] and count < max_loop):
        url = Queue[0]
        print(url)
        Queue= Queue[1:]
        
    
        #Reading HTML text and parsing it
        req = requests.get(url)
        content = req.text            
        s = BeautifulSoup(content, "html.parser")   
        print("soup object")
        
        data=s.find_all("div", class_="result-container")

        #reading information about journals such as title of journal, 
        for info in data:
            #checking author is from CU or not
            a_name=[]   
            a_link=[]
            
            indiv_result={}
            
            a_details= info.find_all("a", class_="link person")
            
            for a in a_details:
                if(a.get('href')!=[]):
                    pro_link = a.get('href')
                    author=a.string

                    a_name.append(author)

                    a_link.append(pro_link)

                    print(author , "-" , pro_link)
                    indiv_result['Authors']=a_name
                    indiv_result['Profile']=a_link
            
            if(a_name!=[]):
                p_name= info.find("h3", class_="title").text
                print(p_name)
                indiv_result['Journal name']=p_name

                p_link = info.find("a", class_="link").get('href')
                print(p_link)
                indiv_result['Journal URL']= p_link

                p_date=info.find("span", class_="date").text
                print(p_date)
                indiv_result['Publishing Date']= p_date

                write_to_file.append(indiv_result)
        
        #Next page link
        next_page=s.find("a", class_="nextLink")
        
        if(next_page!= None):

            next_page_link=next_page.get('href')
            
            next_page_link = baseurl + next_page_link 

            print("Link added to queue for next page is: ", next_page_link)
            
            Queue.append(next_page_link) 

        
        
        #making outputs csv file
        output = open('output.csv', 'w', encoding="utf-8")
        labels=['Authors','Profile','Journal name','Journal URL','Publishing Date']
        
        writer = csv.DictWriter(output, fieldnames=labels)
        writer.writeheader()
        
        for r in write_to_file:
            writer.writerow(r)
        output.close()
        
        count+= 1
    
        


In [89]:
crawler_input=int(input("Press 1 to run crawler and 2 to continue from saved output file"))
if (crawler_input==1):
    webcrawler('https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences',20)
elif (crawler_input==2):
    if(os.path.exists("output.csv")):
        print("Search started from initially saved Crawler optput")
    else: 
        print("You need to crawler first.")
else:
       print("invalid response.")

Press 1 to run crawler and 2 to continue from saved output file3
invalid response.


In [71]:
#Reading csv file and storing contents in dataframe
df=pd.read_csv("output.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Authors          513 non-null    object
 1   Profile          513 non-null    object
 2   Journal name     513 non-null    object
 3   Journal URL      513 non-null    object
 4   Publishing Date  513 non-null    object
dtypes: object(5)
memory usage: 20.2+ KB


In [91]:
#Applying Pre-processing steps
import re
import string
processed_journal = []
for t in df['Journal name']:
    # Removing Unicode
    clean_title = re.sub(r'[^\x00-\x7F]+', '', t)
    # Removing Mentions
    clean_title = re.sub(r'@\w+', '', clean_title)
    # Converting into lowercase
    clean_title = clean_title.lower()
    # Removing punctuations
    clean_title = re.sub(r'[%s]' % re.escape(string.punctuation), '', clean_title)
    #removing stop words
    stop = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    text = stop.sub('', clean_title)
    processed_journal.append(text)
    
print(processed_journal)

['comparison physical activity homebased centrebased pulmonary rehabilitation randomised controlled secondary analysis', 'actual perceived motor competence mediate relationship physical fitness technical skill performance young soccer players', 'acute hormonal response kettlebell swing exercise differs depending load even total work normalized', 'associations selfreported sleep duration mortality employed individuals systematic review metaanalysis', 'blood cancer care resource limited setting covid19 outbreak single center experience sri lanka', 'british soccer academy personnel perceive psychological technicaltactical attributes important contributors development', 'changes joint kinematics dynamic postural stability free restricted arm movements children', 'comparative analysis reported physical activity leisure centres members versus general population spain', 'crosscultural comparison fundamental movement skills 9 10 year old children england china', 'delivery approaches within exe

In [92]:
#Stemming and removing stop words
preprocessed_corpus=[]
for i in processed_journal:
    token_words=word_tokenize(i)
    stem_words=[]
    
    for word in token_words:
      
        if word not in stopwords.words("english"):
            
            if(word.isalpha()):
                
                stemmer = PorterStemmer()
                stem_words.append(stemmer.stem(word))
                doc=' '.join(stem_words)
    
    preprocessed_corpus.append(doc)

print(preprocessed_corpus)

['comparison physic activ homebas centrebas pulmonari rehabilit randomis control secondari analysi', 'actual perceiv motor compet mediat relationship physic fit technic skill perform young soccer player', 'acut hormon respons kettlebel swing exercis differ depend load even total work normal', 'associ selfreport sleep durat mortal employ individu systemat review metaanalysi', 'blood cancer care resourc limit set outbreak singl center experi sri lanka', 'british soccer academi personnel perceiv psycholog technicaltact attribut import contributor develop', 'chang joint kinemat dynam postur stabil free restrict arm movement children', 'compar analysi report physic activ leisur centr member versu gener popul spain', 'crosscultur comparison fundament movement skill year old children england china', 'deliveri approach within exercis referr scheme survey current practic england', 'develop power athlet part practic applic', 'resist sled tow improv physic qualiti elit youth soccer player differ 

In [43]:
#The result (matrix) will become a representation of the documents. 
#we can find the similarity between different documents based on the matrix. 
from sklearn.feature_extraction.text import TfidfVectorizer
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(preprocessed_corpus)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
tfidf = pd.DataFrame(X, index=vectorizer.get_feature_names())

In [44]:
tfidf

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,512
abil,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
absolut,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abund,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
academ,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
academi,0.0,0.000000,0.0,0.0,0.0,0.243868,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young,0.0,0.276335,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
youth,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zimbabw,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zr,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
q1 = input("Enter topic for searching")
preprocessed_query=[]
#for i in q1:
    
token_words=word_tokenize(q1.lower())
stem_words=[]

for word in token_words:

    if word not in stopwords.words("english"):

        if(word.isalpha()):

            stemmer = PorterStemmer()
            stem_words.append(stemmer.stem(word))
            doc=' '.join(stem_words)

preprocessed_query.append(doc)

print(preprocessed_query)
#q1 = [q1]
q_vec = vectorizer.transform(preprocessed_query).toarray().reshape(tfidf.shape[0],)
print(q_vec.shape)

Enter topic for searchingcomparison physical activity homebased centrebased pulmonary rehabilitation randomised controlled secondary analysis
['comparison physic activ homebas centrebas pulmonari rehabilit randomis control secondari analysi']
(1853,)


In [107]:
import math

def calc_inner(v1, v2): # receives two vectors of the same length as lists and returns their inner product
    ans = 0
    for i in range(len(v1)):
        ans += v1[i]*v2[i]
    return ans

def calc_length(v): # receives a vector as a list and returns its length
    tmp = 0
    for x in v:
        tmp += x**2
    #Alternativelt, instead of the above loop, you may write tmp = calc_inner(v,v).
    return math.sqrt(tmp)

def calc_cosine(v1,v2): # receives two vectors of the same length as lists and returns their cosine similarity
    return calc_inner(v1,v2) / (calc_length(v1)*calc_length(v2))
v0 = q_vec

k=[]
for v in range(0,512):
    x=calc_cosine(v0, tfidf[v])
    if(x>0.0):
    
        k.append(x)
    

print(k)


[1.0000000000000002, 0.03727147319390143, 0.14163577233611127, 0.08386702540794326, 0.0358584779784148, 0.11842559678575772, 0.08487491228397132, 0.06800471135253998, 0.041333665655874784, 0.0686505456063959, 0.07279117726836284, 0.045308273723422034, 0.09573868359111012, 0.03915359009088743, 0.056488556394497194, 0.08342414179490266, 0.06745742579026248, 0.0592823886221481, 0.031763911427963816, 0.06804676881813322, 0.04999644271343618, 0.07863937778708498, 0.07462919355400191, 0.028793169511915642, 0.06808897749908431, 0.06156954723164733, 0.06601706895342148, 0.07811627780806091, 0.1627991006870629, 0.06866158814395733, 0.11394981352358347, 0.09562780194540811, 0.10649529753332114, 0.04380665878267491, 0.07199011705481205, 0.1507642477052811, 0.074530138131693, 0.47848346224252214, 0.13967265236354912, 0.07137349955323873, 0.06346888801117333, 0.04263007000570666, 0.09577817137147733, 0.16313499755257435, 0.03231066283755772, 0.030011761848979177, 0.08854277150377776, 0.173550439591

In [108]:
a= (sorted( [(x,i) for (i,x) in enumerate(k)], reverse=True ) [:20] )
print(a)

[(1.0000000000000002, 0), (0.47848346224252214, 37), (0.3242140591101352, 87), (0.306858926770661, 71), (0.2619453277870722, 77), (0.25079253949255464, 76), (0.2145795209328353, 75), (0.19776952184532362, 66), (0.17355043959158575, 47), (0.16313499755257435, 43), (0.1627991006870629, 28), (0.1507642477052811, 35), (0.14163577233611127, 2), (0.13967265236354912, 38), (0.13894003896763393, 52), (0.12999391885641778, 49), (0.11851259626062023, 64), (0.11842559678575772, 5), (0.11394981352358347, 30), (0.10649529753332114, 32)]


In [105]:
if(a==[]):
    print("No related journal found")
else:
    for i in range(len(a)):
        x=a[i][1]
        
        print("Title of Paper:",df.iloc[x,2])
        print("Link of Paper:",df.iloc[x,3])
        print("Date of Paper Published:",df.iloc[x,4])
        print("Author names:",df.iloc[x,0])
        print("Link to Author Profiles:",df.iloc[x,1])
        print("---------------------------------------")


No related journal found


# Task-2: Document Clustering

In [ ]:
pip install feedparser

In [109]:
import feedparser
text=[]

Politics = feedparser.parse("http://feeds.bbci.co.uk/news/politics/rss.xml")
Education = feedparser.parse("http://feeds.bbci.co.uk/news/education/rss.xml")
Science = feedparser.parse("http://feeds.bbci.co.uk/news/science_and_environment/rss.xml")

for i in Politics.entries:
    x= i.summary
    text.append(x)

for i in Education.entries:
    x= i.summary
    text.append(x)

for i in Science.entries:
    x= i.summary
    text.append(x)

print(len(text))
print(text)

144
['The Labour MP accuses the government of stoking a culture war that will harm anyone "slightly different".', 'The south-east London election was prompted by the death of former MP James Brokenshire.', 'But government ministers are accused of giving mixed messages on events over the festive period.', 'Alex Bourne says the former health secretary had no involvement is his company winning NHS work.', 'Grenfell survivors and government minister Michael Gove criticise the F1 team over a sponsorship deal.', 'Maroš Šefčovič says it is "time to get medicines across the finish line" but again no deal is struck.', 'The Barking MP held ministerial posts and chaired the powerful Public Accounts Committee.', 'Sir Desmond Swayne raises concerns about changes to the code of conduct governing MPs behaviour.', 'The addresses of more than 1,000 people, including Sir Elton John and Gabby Logan, were shared online.', 'The government acted too slowly to stop fraudsters exploiting the Covid loan scheme

In [110]:
#Preprocessing
ps = PorterStemmer()
clean_text = []
for i in text:
    word_tokens = word_tokenize(i.lower())
    n = ""
    for w in word_tokens:
        if w not in stopwords.words("english"):
            if (w.isalpha()):
                n += ps.stem(w) + " "
    clean_text.append(n)
print(clean_text)

['labour mp accus govern stoke cultur war harm anyon slightli differ ', 'london elect prompt death former mp jame brokenshir ', 'govern minist accus give mix messag event festiv period ', 'alex bourn say former health secretari involv compani win nh work ', 'grenfel survivor govern minist michael gove criticis team sponsorship deal ', 'maroš šefčovič say time get medicin across finish line deal struck ', 'bark mp held ministeri post chair power public account committe ', 'sir desmond swayn rais concern chang code conduct govern mp behaviour ', 'address peopl includ sir elton john gabbi logan share onlin ', 'govern act slowli stop fraudster exploit covid loan scheme report say ', 'report suggest us decis maintain tariff uk steel link brexit reject ', 'sir jeffrey donaldson say covid joke matter respons sammi wilson tweet ', 'lord mckenzi luton describ one town ardent servant ', 'sir richard lees step role manchest citi council leader year ', 'three conserv includ sir iain duncan smith g

In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
data = vectorizer.fit_transform(clean_text)
print(data.todense())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [112]:
from sklearn.cluster import KMeans
K = 3
model = KMeans(n_clusters=K)
model.fit(data)

print(model.labels_)

[2 2 2 2 2 2 2 2 0 2 0 2 0 0 0 1 0 0 2 2 2 2 2 0 2 0 2 0 2 2 0 0 2 2 1 0 0
 0 2 0 0 2 2 0 0 0 2 0 0 0 0 2 2 0 0 2 0 2 2 2 0 0 0 0 2 0 0 0 0 0 1 2 0 2
 0 1 2 0 0 0 0 2 2 2 2 0 0 0 2 1 0 2 2 2 2 0 0 0 0 0 0 2 2 2 0 0 2 1 1 0 2
 0 2 0 1 2 1 0 1 2 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 2]


In [115]:
#Taking input from user
q = input("Enter a string for which you need to know cluster: ")

#pre-processing 
word_tokens = word_tokenize(q)
x = ""
for w in word_tokens:
    if w not in stopwords.words("english"):
        x += ps.stem(w) + " "
print(x)   

#predicting cluster
Y = vectorizer.transform([x])
res = model.predict(Y)
print("cluster is: ", res)

Enter a string for which you need to know cluster: face mask covering are important
face mask cover import 
cluster is:  [2]
